# Importaciones

In [1]:
import os
import time

import cv2
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt

# Funciones

In [2]:
def get_mp_resources():
    return mp.solutions.holistic, mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversión de color de RGB a RGB
    image.flags.writeable = False                  # Se bloquea la escritura de la imagen
    results = model.process(image)                 # Se realiza la predicción
    image.flags.writeable = True                   # Se habilita la escritura de la imagen
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Conversión de color RGB a RGB
    return image, results

In [3]:
def draw_styled_landmarks(image, results, mp_holistic, mp_drawing):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                            )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                            )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                            )

In [6]:
def start_video_capture(mp_holistic, mp_drawing):
    captura = cv2.VideoCapture(0)

    # Acceso/configuración al modulo de mediapipe
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while captura.isOpened():
            # Lectura via camara
            _, frame = captura.read()

            # Realizar detecciones
            image, results = mediapipe_detection(frame, holistic)

            # Dibujar landmarks
            draw_styled_landmarks(image, results, mp_holistic, mp_drawing)

            image = cv2.flip(image, 1)

            # Mostrar en pantalla
            cv2.imshow('OpenCV', image)

            # Salida
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        captura.release()
        cv2.destroyAllWindows()
        return results

In [7]:
def extract_keypoints(results):
    landmarks_data = {
        "left_hand": np.array(
            [
                [res.x, res.y, res.z]
                for res in results.left_hand_landmarks.landmark
            ]).flatten()
            if results.left_hand_landmarks
            else np.zeros(21*3),

        "right_hand": np.array(
            [
                [res.x, res.y, res.z]
                for res in results.right_hand_landmarks.landmark
            ]).flatten()
            if results.right_hand_landmarks
            else np.zeros(21*3),

        "face": np.array(
            [
                [res.x, res.y, res.z]
                for res in results.face_landmarks.landmark
            ]).flatten()
            if results.face_landmarks
            else np.zeros(468*3),

        "pose": np.array(
            [
                [res.x, res.y, res.z, res.visibility]
                for res in results.pose_landmarks.landmark
            ]).flatten()
            if results.pose_landmarks
            else np.zeros(33*4)
    }
    return landmarks_data, np.concatenate([
        landmarks_data["pose"],
        landmarks_data["face"],
        landmarks_data["left_hand"],
        landmarks_data["right_hand"],
    ])